In [ ]:
import pandas as pd
import csv
def generate_prompt(question):
    prompt_template = """
    Answer the following question with a detailed explanation:
    Question: {question}
    """
    return prompt_template.format(question=question)


In [ ]:
import pandas as pd
import openai
import requests
import openai
import json
import os
import csv
from openai import OpenAI

client = OpenAI(api_key=api_key)
# -------------------------------
# Set your API keys here
# -------------------------------
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"
DEEPSEEK_API_KEY = "YOUR_DEEPSEEK_API_KEY"



def call_openai_api_batch(prompts):
    """
    Call the OpenAI API in batch mode using a hypothetical batch endpoint.
    Adjust the URL and payload as required by the API's documentation.
    """
    # .jsonl file preparation
    batch_requests = []
    for i, prompt in enumerate(prompts):
        request_item = {
            "custom_id": f"request-{i+1}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "o1",
                "messages": [
                    {"role": "user", "content": prompt}
                ],
            }
        }
        batch_requests.append(request_item)
    
    # Write the batch requests to a .jsonl file
    batch_file_path = "openai_batch_requests.jsonl"
    with open(batch_file_path, "w") as f:
        for req in batch_requests:
            json_line = json.dumps(req)
            f.write(json_line + "\n")
    
    # # Read the contents of the file to use as the payload
    # with open(batch_file_path, "r") as f:
    #     batch_data = f.read()

    batch_input_file = client.files.create(
    file=open("openai_batch_requests.jsonl", "rb"),
    purpose="batch"
    )
    print("Batch file uploaded successfully.",batch_input_file)
    batch_input_file_id = batch_input_file.id
    
    response = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
    )
    print("Batch job created successfully.",batch_input_file_id)
    print("Batch Object",response)
    # Wait for the batch job to complete
    
    


    




### check status of openai batch calling

In [23]:
batch = client.batches.retrieve("batch_67ed1bc062848190934e090d59bf7491")
batch

Batch(id='batch_67ed1bc062848190934e090d59bf7491', completion_window='24h', created_at=1743592384, endpoint='/v1/chat/completions', input_file_id='file-5ZrLE12oNxSULCKWfCMb6p', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1743592584, error_file_id=None, errors=None, expired_at=None, expires_at=1743678784, failed_at=None, finalizing_at=1743592583, in_progress_at=1743592385, metadata={'description': 'nightly eval job'}, output_file_id='file-Jj4hfBBfd9GXaEVMoihXPw', request_counts=BatchRequestCounts(completed=10, failed=0, total=10))

### Openai API result 

In [ ]:
file_response = client.files.content("file-Jj4hfBBfd9GXaEVMoihXPw")
print(file_response.text)

#save the response to a file
with open("batch_response.json", "w") as f:
    f.write(file_response.text)

{"id": "batch_req_67ed1c877bcc8190b6085598b3cd7bc3", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "03b8f812da8919c6f8346c1a626f75a7", "body": {"id": "chatcmpl-BHqJmlmf7YZKZirPhj7t7TwERUGBp", "object": "chat.completion", "created": 1743592386, "model": "o1-2024-12-17", "choices": [{"index": 0, "message": {"role": "assistant", "content": "It appears the prompt contains \u201cnan\u201d as the question. In many contexts\u2014especially in computing\u2014\u201cNaN\u201d is an acronym for \u201cNot a Number.\u201d This special placeholder is used to represent a value that is mathematically or computationally undefined. Below is a breakdown of what \u201cNaN\u201d typically means and where you might encounter it:\n\n1. Meaning of NaN:\n   \u2022 \u201cNaN\u201d stands for \u201cNot a Number,\u201d and it usually appears in floating-point arithmetic.  \n   \u2022 In many programming languages (such as JavaScript, Python, or C), NaN is the result of certain operation

In [ ]:
def call_gemini_api_batch(prompts):
    """
    Call the Gemini API in batch mode using a hypothetical batch endpoint.
    Adjust the URL, headers, and payload according to the Gemini API documentation.
    """
        # .jsonl file preparation
    batch_requests = []
    for i, prompt in enumerate(prompts):
        request_item = {
            "request": {
                "contents":
                    [
                        {
                            "role": "user",
                            "parts":[
                                {
                                    "text": prompt
                                }
                            ]
                        }
                    ]
                
            }
        }
        batch_requests.append(request_item)

    # Write the batch requests to a .jsonl file
    batch_file_path = "gemini_batch_requests.jsonl"
    with open(batch_file_path, "w") as f:
        for req in batch_requests:
            json_line = json.dumps(req)
            f.write(json_line + "\n")

    import time

    from google import genai

    client = genai.Client(api_key='my-api-key')


    job = client.batches.create(
    model='gemini-1.5-flash-002',
    src='bq://my-project.my-dataset.my-table',
    )
    print("Batch job created successfully.",job)

    

    


In [ ]:
def claude_batch(prompts):
    """
    Call the Claude API in batch mode using a hypothetical batch endpoint.
    Adjust the URL, headers, and payload according to the Claude API documentation.
    """
    import anthropic
    from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
    from anthropic.types.messages.batch_create_params import Request

    client = anthropic.Anthropic(api_key="YOUR_CLAUDE_API_KEY")
    # .jsonl file preparation
    batch_requests = []
    for i, prompt in enumerate(prompts):
        request_item = Request(
            custom_id=f"request-{i+1}",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                messages=[
                {
                    "role": "user", 
                    "content": prompt
                }]
            )

        )
        batch_requests.append(request_item)

    # # Write the batch requests to a .jsonl file
    # batch_file_path = "claude_batch_requests.jsonl"
    # with open(batch_file_path, "w") as f:
    #     for req in batch_requests:
    #         json_line = json.dumps(req)
    #         f.write(json_line + "\n")

    message_batch = client.messages.batches.create(
        requests=batch_requests
    )
    print("Batch job created successfully.",message_batch)

In [ ]:

# -------------------------------
# Single API Call Function
# -------------------------------

from openai import OpenAI

deepseek_client = OpenAI(api_key="<DeepSeek API Key>", base_url="https://api.deepseek.com")
def call_deepseek_api(prompt):
    """
    Call the Deepseek API for a single prompt.
    Adjust the URL, headers, and payload according to Deepseek API documentation.
    """
    try:
        response = deepseek_client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt}
            ],
        )
        answer = response['choices'][0]['message']['content']
    except Exception as e:
        answer = f"Error: {str(e)}"
    return answer

def together_api_call(prompt):
    """
    Call the Together API for a single prompt.
    
    """
    from together import Together

    client = Together()

    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1",
        messages=[{"role": "user", "content": prompt}],
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content
    

# -------------------------------
# Process CSV and Record Answers
# -------------------------------
def process_questions(input_csv, output_csv):
    """
    Read the CSV file, generate prompts for each question, batch call OpenAI and Gemini APIs,
    call Deepseek API for each prompt individually, and record all answers in new columns.
    """
    df = pd.read_csv(input_csv)
    df['deepseek_answer'] = ""
    
    # Generate prompts for each row and store in a list (preserving order)
    prompts = []
    for index, row in df.iterrows():
        question = row['User Query']
        prompt = generate_prompt(question)
        prompts.append(prompt)

    
    # Single prompt calling for Deepseek (or other models that do not support batch calls)
    deepseek_answers = []
    print("Calling Deepseek API for each prompt...")
    for prompt in prompts:
        ans = call_deepseek_api(prompt)
        deepseek_answers.append(ans)
    
    df['deepseek_answer'] = deepseek_answers
    
    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# -------------------------------
# Main Execution
# -------------------------------



In [19]:
input_csv = "Copy of [Deccan AI] PCM samples - Mathematics.csv"   # Replace with your input CSV file path
output_csv = "output_answers.csv"     # Replace with your desired output CSV file path

df = pd.read_csv(input_csv)


# Generate prompts for each row and store in a list (preserving order)
prompts = []
for index, row in df.iterrows():
    question = row['User Query']
    prompt = generate_prompt(question)
    prompts.append(prompt)

# Call the OpenAI API in batch mode
print("Calling OpenAI batch API...")
call_openai_api_batch(prompts)
# print("Calling Gemini batch API...")
# call_gemini_api_batch(prompts)




Calling OpenAI batch API...
Batch file uploaded successfully. FileObject(id='file-5ZrLE12oNxSULCKWfCMb6p', bytes=6862, created_at=1743592383, filename='openai_batch_requests.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)
Batch job created successfully. file-5ZrLE12oNxSULCKWfCMb6p
Batch Object Batch(id='batch_67ed1bc062848190934e090d59bf7491', completion_window='24h', created_at=1743592384, endpoint='/v1/chat/completions', input_file_id='file-5ZrLE12oNxSULCKWfCMb6p', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1743678784, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [16]:
# from openai import OpenAI
# client = OpenAI()

batch = client.batches.retrieve("file-SkjSkiYhkEuFfTSkVi3Rxs")
print(batch)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'batch_id': 'file-SkjSkiYhkEuFfTSkVi3Rxs'. Expected an ID that begins with 'batch'.", 'type': 'invalid_request_error', 'param': 'batch_id', 'code': 'invalid_value'}}

# Evaluation 

In [ ]:
import pandas as pd
import openai

# Set your OpenAI API key
openai.api_key = 'YOUR_API_KEY_HERE'

def judge_answer(question, gold_answer, model_answer):
    """
    Constructs a prompt to have OpenAI judge whether the model_answer matches the gold_answer.
    Returns "Correct" if they match, otherwise "Incorrect".
    """
    prompt = f"""
    You are an expert evaluator. Here is the question, the correct (gold) answer, and a model's answer.
    
    Question: {question}
    Gold Answer: {gold_answer}
    Model Answer: {model_answer}
    
    Based on the gold answer, determine if the model answer is correct. 
    Answer only with "Correct" if they match or "Incorrect" if they do not.
    """
    
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=10,  # Small output since we expect a binary answer
        temperature=0  # Deterministic output
    )
    
    return response.choices[0].text.strip()

# Load your CSV file (update the path as needed)
df = pd.read_csv("your_file.csv")

# Apply the judging function to each row for both Gemini and Deepseek responses
df['gemini_judgment'] = df.apply(
    lambda row: judge_answer(row['question'], row['gold label'], row['gemini_response']), axis=1
)
df['deepseek_judgment'] = df.apply(
    lambda row: judge_answer(row['question'], row['gold label'], row['deepseek response']), axis=1
)

# Save the updated DataFrame with judgments to a new CSV file
df.to_csv("judged_results.csv", index=False)

print("Judgment complete. Results saved to judged_results.csv.")
